# Data Wrangling: Join, Combine, and Reshape
## Data-oriented programming paradigms 2021W

---
<img src="http://ifs.tuwien.ac.at/~piroi/logos/logo-tuwien-informatics.png" width="300">

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(12345)

## Hierarchical Indexing

In [6]:
s_data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [7]:
s_data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [8]:
s_data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

labels:
[0, 0, 0, 1, 1, 2, 2, 3, 3] -> 0-2: a, 3-4:b etc.
[0, 1, 2, 0, 2, 0, 1, 1, 2] -> 1->0->'1', 2->1->'2"

Partial indexing makes it easy to select subsets of data:

In [9]:
s_data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [10]:
s_data['b']

1   -0.555730
3    1.965781
dtype: float64

In [11]:
s_data['b':'c']

b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
dtype: float64

In [12]:
s_data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [13]:
# Selecting individual rows by index
s_data.loc[['b', 'd']]

b  1   -0.555730
   3    1.965781
d  2    0.281746
   3    0.769023
dtype: float64

In [14]:
s_data.loc[:, 2]

a    0.478943
c    0.092908
d    0.281746
dtype: float64

Rarrange data into a DataFrame:

In [15]:
s_data.unstack().stack()

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

We'll get back to that...

### Hierarchical indexes for DataFrames

In [16]:
df = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['baz', 'baz', 'foo', 'foo'], [1, 2, 1, 2]],
                     columns=[['A', 'A', 'B'],
                              ['x', 'y', 'z']])
df

A       B
       x   y   z
baz 1  0   1   2
    2  3   4   5
foo 1  6   7   8
    2  9  10  11

In [17]:
# Access columns by top level index
df['A']

x   y
baz 1  0   1
    2  3   4
foo 1  6   7
    2  9  10

### Reordering and Sorting Levels

In [18]:
# Let's assign names to levels first..
df.index.names = ['key1', 'key2']
df

A       B
           x   y   z
key1 key2           
baz  1     0   1   2
     2     3   4   5
foo  1     6   7   8
     2     9  10  11

In [19]:
# Now we can swap them
df.swaplevel('key1', 'key2')

A       B
           x   y   z
key2 key1           
1    baz   0   1   2
2    baz   3   4   5
1    foo   6   7   8
2    foo   9  10  11

In [20]:
# Sort by specific index level
df.sort_index(level=1)

df.swaplevel(0, 1).sort_index(level=0)

A       B
           x   y   z
key2 key1           
1    baz   0   1   2
     foo   6   7   8
2    baz   3   4   5
     foo   9  10  11

### Summary Statistics by Level

In [21]:
df

A       B
           x   y   z
key1 key2           
baz  1     0   1   2
     2     3   4   5
foo  1     6   7   8
     2     9  10  11

In [22]:
df.sum(level='key1')

A       B
       x   y   z
key1            
baz    3   5   7
foo   15  17  19

### Indexing with a DataFrame's columns

In [23]:
df = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
df

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [24]:
df2 = df.set_index(['c', 'd'])
df2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

#### Resetting an index

In [25]:
df2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## Combining and Merging Datasets

1. `pd.merge` implements database join operations, i.e., connects rows in DataFrames based on one or more keys. 
1. `pd.concat` “stacks” together objects along an axis.
1. `combine_first` is an instance method that enables splicing together overlapping data to fill in missing values in one object with values from another.

### Database-Style DataFrame Joins
#### Inner join

In [26]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
display(df1, df2)

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,key,data2
0,a,0
1,b,1
2,d,2


In [27]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Uses the overlapping column names as keys.

Join columns can also be specified explicitly:

In [28]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
display(df3, df4)
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,rkey,data2
0,a,0
1,b,1
2,d,2


,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


By default, merge does an inner join (→ C and D columns missing). If we want outer..

#### Outer join

In [29]:
display(df1, df2)
pd.merge(df1, df2, how='outer')

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,key,data2
0,a,0
1,b,1
2,d,2


,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


#### Left and right joins

Many-to-many merges that form Cartesian product:

In [30]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
display(df1,df2)

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [31]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


#### Merge with multiple keys

In [32]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
display(left, right)

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [33]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


We can automatically append suffixes to column names to make them easier to distinguish: 

In [34]:
pd.merge(left, right, on='key1')
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merging on Index

Specify *left_Index=True* and/or *right_index=True*

In [35]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
display(left1, right1)

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


,group_val
a,3.5
b,7.0


In [36]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [37]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


#### Merging on hierarchical indices

In [38]:
lefth = pd.DataFrame({'key1': ['Vienna', 'Vienna', 'Vienna',
                               'Berlin', 'Berlin'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Berlin', 'Berlin', 'Vienna', 'Vienna',
                              'Vienna', 'Vienna'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
display(lefth,righth)

,key1,key2,data
0,Vienna,2000,0.0
1,Vienna,2001,1.0
2,Vienna,2002,2.0
3,Berlin,2001,3.0
4,Berlin,2002,4.0


event1  event2
Berlin 2001       0       1
       2000       2       3
Vienna 2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [39]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Vienna,2000,0.0,4,5
0,Vienna,2000,0.0,6,7
1,Vienna,2001,1.0,8,9
2,Vienna,2002,2.0,10,11
3,Berlin,2001,3.0,0,1


In [40]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Vienna,2000,0.0,4.0,5.0
0,Vienna,2000,0.0,6.0,7.0
1,Vienna,2001,1.0,8.0,9.0
2,Vienna,2002,2.0,10.0,11.0
3,Berlin,2001,3.0,0.0,1.0
4,Berlin,2002,4.0,NaN,NaN
4,Berlin,2000,NaN,2.0,3.0


#### Joining on indices on both sides

In [41]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Vienna', 'Graz'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Innsbruck', 'Salzburg'])
display(left2,right2)

,Vienna,Graz
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


,Innsbruck,Salzburg
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [42]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

# or more conviently:
left2.join(right2, how='outer')

,Vienna,Graz,Innsbruck,Salzburg
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [43]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


### Concatenating Along an Axis

In [44]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
display(s1,s2,s3)

a    0
b    1
dtype: int64

c    2
d    3
e    4
dtype: int64

f    5
g    6
dtype: int64

In [45]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

By default, *concat* glues together the values and indexes along axis=0. If we pass axis=1, the result will be a DataFrame (axis=1 is the columns) → outer join of indexes

In [46]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


#### Inner join 

In [47]:
display(s1,s2,s3)

a    0
b    1
dtype: int64

c    2
d    3
e    4
dtype: int64

f    5
g    6
dtype: int64

In [48]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [51]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [55]:
pd.concat( [s1, s4], axis = 1, join_axes = [['a', 'c', 'b', 'e']])

TypeError: concat() got an unexpected keyword argument 'join_axes'

In [53]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [56]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [57]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [58]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


A more complex example with data frames, where we want to be able to identify the concatenated pieces: 

In [59]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])
display(df1,df2)

,one,two
a,0,1
b,2,3
c,4,5


,three,four
a,5,6
c,7,8


In [60]:
pd.concat([df1, df2], axis=1, keys=['df1_level', 'df2_level'])

df1_level     df2_level     
        one two     three four
a         0   1       5.0  6.0
b         2   3       NaN  NaN
c         4   5       7.0  8.0

In [61]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [62]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [63]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
display(df1,df2)

,a,b,c,d
0,1.246435,1.007189,-1.296221,0.274992
1,0.228913,1.352917,0.886429,-2.001637
2,-0.371843,1.669025,-0.438570,-0.539741


,b,d,a
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614


In [64]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,1.246435,1.007189,-1.296221,0.274992
1,0.228913,1.352917,0.886429,-2.001637
2,-0.371843,1.669025,-0.438570,-0.539741
3,-1.021228,0.476985,NaN,3.248944
4,0.302614,-0.577087,NaN,0.124121


### Combining Data with Overlap

In [65]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
display(a,b)

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

On serieses, where can be used to implement if-else expressions 

In [66]:
# only take data from b if there is no data in a
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

Series has a combine_first method, which performs the equivalent of this operation along with pandas’s usual data alignment logic:

In [67]:
b.combine_first(a)

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

With DataFrames, *combine_first* does the same thing column by column

In [68]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
display(df1,df2)

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [69]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## Reshaping and Pivoting

### Reshaping with Hierarchical Indexing

In [70]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Vienna', 'Tyrol'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

number,one,two,three
state,,,
Vienna,0,1,2
Tyrol,3,4,5


*stack* pivots columns in the data into the rows

In [71]:
result = data.stack()
result

state   number
Vienna  one       0
        two       1
        three     2
Tyrol   one       3
        two       4
        three     5
dtype: int32

In [72]:
result.unstack() # innermost level by default

number,one,two,three
state,,,
Vienna,0,1,2
Tyrol,3,4,5


In [73]:
result.unstack(0)
result.unstack('state')

state,Vienna,Tyrol
number,,
one,0,3
two,1,4
three,2,5


Note that unstacking may introduce missing data..

In [74]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
display(data2)
data2.unstack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


Stacking filters out missing data by default..

In [75]:
display(data2.unstack())
display(data2.unstack().stack())
display(data2.unstack().stack(dropna=False)) # we can override this

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [76]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
display(df)
df.unstack('state')

side           left  right
state  number             
Vienna one        0      5
       two        1      6
       three      2      7
Tyrol  one        3      8
       two        4      9
       three      5     10

side     left        right      
state  Vienna Tyrol Vienna Tyrol
number                          
one         0     3      5     8
two         1     4      6     9
three       2     5      7    10

In [77]:
df.unstack('state').stack('side')

state         Tyrol  Vienna
number side                
one    left       3       0
       right      8       5
two    left       4       1
       right      9       6
three  left       5       2
       right     10       7

### Pivoting “Long” to “Wide” Format

In [81]:
#NOTE: In Windows the next command will give an error.
!head -n 3 macrodata.csv

'head' is not recognized as an internal or external command,
operable program or batch file.


In [83]:
#use "more" instead of "head" if you are on a Windows machine
!more macrodata.csv

"year","quarter","realgdp","realcons","realinv","realgovt","realdpi","cpi","m1","tbilrate","unemp","pop","infl","realint"
1959,1,2710.349,1707.4,286.898,470.045,1886.9,28.980,139.7,2.82,5.8,177.146,0,0
1959,2,2778.801,1733.7,310.859,481.301,1919.7,29.150,141.7,3.08,5.1,177.830,2.34,0.74
1959,3,2775.488,1751.8,289.226,491.260,1916.4,29.350,140.5,3.82,5.3,178.657,2.74,1.09
1959,4,2785.204,1753.7,299.356,484.052,1931.3,29.370,140,4.33,5.6,179.386,0.27,4.06
1960,1,2847.699,1770.5,331.722,462.199,1955.5,29.540,139.6,3.50,5.2,180.007,2.31,1.19
1960,2,2834.390,1792.9,298.152,460.400,1966.1,29.550,140.2,2.68,5.2,180.671,0.14,2.55
1960,3,2839.022,1785.8,296.375,474.676,1967.8,29.750,140.9,2.36,5.6,181.528,2.7,-0.34
1960,4,2802.616,1788.2,259.764,476.434,1966.6,29.840,141.1,2.29,6.3,182.287,1.21,1.08
1961,1,2819.264,1787.7,266.405,475.854,1984.5,29.810,142.1,2.37,6.8,182.992,-0.4,2.77
1961,2,2872.005,1814.3,286.246,480.328,2014.4,29.920,142.9,2.29,7,183.691,1.47,0.81
1961,3,2918.419,1823.1,310.2

In [85]:
s_data = pd.read_csv('macrodata.csv')

periods = pd.PeriodIndex(year = s_data.year, quarter = s_data.quarter,
                         name = 'date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name = 'item')
s_data = s_data.reindex(columns = columns)
s_data.index = periods.to_timestamp('D', 'end')
ldata = s_data.stack().reset_index().rename(columns = {0: 'value'})

In [86]:
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [87]:
# DataFrame.pivot(index=None, columns=None, values=None)[source]
pivoted = ldata.pivot('date', 'item', 'value')
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


In [88]:
# Adding a column
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,0.523772
1,1959-03-31 23:59:59.999999999,infl,0.000,0.000940
2,1959-03-31 23:59:59.999999999,unemp,5.800,1.343810
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-0.713544
4,1959-06-30 23:59:59.999999999,infl,2.340,-0.831154
5,1959-06-30 23:59:59.999999999,unemp,5.100,-2.370232
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,-1.860761
7,1959-09-30 23:59:59.999999999,infl,2.740,-0.860757
8,1959-09-30 23:59:59.999999999,unemp,5.300,0.560145
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,-1.265934


In [89]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


In [90]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.000940  0.523772   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.831154 -0.713544   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.860757 -1.860761   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  0.119827 -1.265934   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -2.359419  0.332883   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -0.970736 -1.541996   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6  0.377984  0.286350   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.343810  
1959-06-30 23:59:59.999999999 -2.370232  
1959-09-30 23:59:59.999999999  0.560145  
1959-12-31 23:59:59.999999999 -1.063512  
1960-03-31 23:59:59.999999999 -0.199543  
1960-06-30 23:59:59.999999999 -1.307030  
1960-09-30 23:59:59.999999999 -0.753887

### Pivoting “Wide” to “Long” Format

In [91]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [92]:
# pandas.melt(frame, id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None)[source]
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [93]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [18]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [94]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [95]:
pd.melt(df, value_vars=['A', 'B', 'C'])
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
